In [1]:
#!pip install polars
%pip install pandas_profiling
%pip install pydantic
import os
import json
import polars as pl 
import pandas as pd
import pandas_profiling
import geopandas as gpd
from ydata_profiling import ProfileReport
import numpy as np

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lektor 3.3.10 requires Babel, which is not installed.
lektor 3.3.10 requires Flask, which is not installed.
lektor 3.3.10 requires mistune<2,>=0.7.0, which is not installed.
lektor 3.3.10 requires python-slugify, which is not installed.
lektor 3.3.10 requires Werkzeug<2.4, which is not installed.
altair 5.0.1 requires jsonschema>=3.0, which is not installed.
altair 5.0.1 requires toolz, which is not installed.
autoviz 0.1.730 requires bokeh~=2.4.2, which is not installed.
autoviz 0.1.730 requires fsspec>=0.8.3, which is not installed.
autoviz 0.1.730 requires jupyter, which is not installed.
autoviz 0.1.730 requires nltk, which is not installed.
autoviz 0.1.730 requires panel>=0.12.6, which is not installed.
autoviz 0.1.730 requires scikit-learn, which is not installed.
bqplot 0.12.40 requires ipywidgets<9,>=7.5.0


                                              0.0/2.1 MB ? eta -:--:--
                                              0.0/2.1 MB ? eta -:--:--
                                              0.0/2.1 MB ? eta -:--:--
                                              0.0/2.1 MB 163.8 kB/s eta 0:00:13
                                              0.0/2.1 MB 196.9 kB/s eta 0:00:11
     --                                       0.1/2.1 MB 561.1 kB/s eta 0:00:04
     -----                                    0.3/2.1 MB 1.1 MB/s eta 0:00:02
     -------------                            0.7/2.1 MB 2.3 MB/s eta 0:00:01
     -------------------------                1.3/2.1 MB 3.7 MB/s eta 0:00:01
     ------------------------------------     1.9/2.1 MB 4.7 MB/s eta 0:00:01
     ---------------------------------------  2.1/2.1 MB 4.8 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 4.5 MB/s eta 0:00:00
                                              0.0/145.3 kB ? eta -:--:--
     

c:\Users\toast\.conda\envs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\toast\AppData\Local\Temp\ipykernel_18256\3318085536.py:8: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [2]:

# Installs the Polars library for working with tabular data
# Imports libraries for working with different data file types and generating data profiling reports
# Loops through all files in the input directory
# Determines file type and reads into appropriate DataFrame 
# Generates a ProfileReport for key info like data types, missing values
# Extracts schema info, missing data stats, and other useful info
# Saves results to a JSON file for later analysis


# Gather schema information
inspection_results = []

directory = r"C:\Users\toast\Downloads\Climate_Data\Tabular\test"
output_folder = r"C:\Users\toast\Downloads\Climate_Data\Tabular\test\profiles"
# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(directory):
    if file.endswith((".csv", ".xlsx", ".geojson", ".shp")):
        file_path = os.path.join(directory, file)
        print(f"Examining file: {file}")
        
        # Determine the file format and use appropriate library
        if file.endswith((".csv", ".xlsx")):
            try:
                if file.endswith(".csv"):
                    df = pd.read_csv(file_path)
                else:
                    df = pd.read_excel(file_path)
                
                # Generate a data profiling report
                profile = ProfileReport(df)
                # Save the profile report with the correct file extension
                output_file_name = f"profile_{os.path.splitext(file)[0]}.html"  # Corrected the extension
                profile.to_file(os.path.join(output_folder, output_file_name))
                
                # Gather schema information
                schema_info = df.dtypes.to_dict()
                
                # Check for missing data
                missing_data = df.isnull().sum().to_dict()
                
                # Extract useful information (customize as needed)
                useful_info = {
                    "Total Rows": len(df),
                    "Total Columns": len(df.columns),
                    "Column Names": df.columns.tolist(),
                    "Profile Report": output_file_name                 ,
                    "Data Types": schema_info,
                    "Missing Data": missing_data,

                    # Add more information as needed
                }
                
            except Exception as e:
                # Handle any errors when reading the file
                schema_info = {}
                missing_data = {}
                useful_info = {"Error": str(e)}
        
        elif file.endswith((".geojson", ".shp")):
            try:
                if file.endswith(".geojson"):
                    df = gpd.read_file(file_path)
                else:
                    df = gpd.read_file(file_path)
                
                # Gather schema information
                schema_info = dict(df.dtypes)
                
                # Check for missing data
                missing_data = df.isnull().sum().to_dict()
                
                # Extract useful information (customize as needed)
                useful_info = {
                    "Total Rows": len(df),
                    "Total Columns": len(df.columns),
                    "Column Names": df.columns.tolist(),
                    "Data Types": schema_info,
                    "Missing Data": missing_data,

                    # Add more information as needed
                }
            except Exception as e:
                # Handle any errors when reading the file
                schema_info = {}
                missing_data = {}
                useful_info = {"Error": str(e)}
        
        # Store the inspection results in a dictionary
        result = {
            "File Name": file,
            "Missing Data": missing_data,
            "Schema Info": schema_info,
            "Useful Info": useful_info,
        }
        
        # Append the result to the list
        inspection_results.append(result)
        print("Inspection complete.")
        print("-----")

        # Save the profile report to disk
        if "Profile Report" in useful_info:
            profile_path = useful_info["Profile Report"]
            if os.path.exists(profile_path):
                os.remove(profile_path)
            if os.path.exists(f"profile_{file}.html"):
                if os.path.exists(profile_path):
                    os.remove(profile_path)
                os.rename(f"profile_{file}.html", profile_path)

# Save the inspection results to a JSON file
output_file = r"C:\Users\toast\Downloads\Climate_Data\inspection_results.json"

def convert_dtype(obj):
    if isinstance(obj, pd.Timestamp):
        return obj.isoformat()
    else:
        return str(obj)

with open(output_file, "w") as json_file:
    json.dump(inspection_results, json_file, default=convert_dtype)

print(f"Inspection results saved to {output_file}")



Examining file: Neighbourhoods_and_Wards_20231017.csv


Summarize dataset:  11%|█         | 1/9 [00:00<00:01,  5.98it/s, Describe variable:Neighbourhood Number]     

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 85.84it/s]


Inspection complete.
-----
Examining file: Root_for_Trees_Inventory_20231022.csv


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 15.87it/s]


Inspection complete.
-----
Examining file: Tree_Insects___Other_Pests_20231017.csv


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 55.56it/s]


Inspection complete.
-----
Examining file: Vacant_Land_-_Industrial_20231017.csv


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 32.30it/s]

Inspection complete.
-----
Inspection results saved to C:\Users\toast\Downloads\Climate_Data\inspection_results.json


In [21]:
report = pd.DataFrame(inspection_results)

In [22]:
report.to_csv(r"C:\Users\toast\Downloads\Climate_Data\inspection_results.csv")